In [1]:
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import root_mean_squared_error
import os, io
import torch

%matplotlib inline


plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
movies = pd.read_csv("/content/drive/MyDrive/CMPE256/256_project/Original_datasets/Netflix_Dataset_Movie.csv")
titles= pd.read_csv("/content/drive/MyDrive/CMPE256/256_project/Original_datasets/title.basics.tsv", sep='\t')

<ipython-input-3-e5b05824dc8c>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles= pd.read_csv("/content/drive/MyDrive/CMPE256/256_project/Original_datasets/title.basics.tsv", sep='\t')


In [4]:
titles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
titles=titles.drop(columns=['tconst','endYear'])

In [7]:
titles['isAdult']=titles['isAdult'].replace('\\N',np.nan)
titles['startYear']=titles['startYear'].replace('\\N',np.nan)
titles['runtimeMinutes']=titles['runtimeMinutes'].replace('\\N',np.nan)
titles['genres']=titles['genres'].replace('\\N',np.nan)

In [8]:
titles.isna().sum()

,0
titleType,0
primaryTitle,19
originalTitle,19
isAdult,1
startYear,1417009
runtimeMinutes,7661475
genres,500885


In [15]:
titles['runtimeMinutes']=pd.to_numeric(titles['runtimeMinutes'], errors='coerce')

In [21]:
titles = titles[titles.titleType != 'videoGame']

In [30]:
titles['startYear']=titles['startYear'].replace(np.NaN,0)
titles['startYear']=titles['startYear'].astype('int64')

<ipython-input-30-dc3bccf2ab70>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titles['startYear']=titles['startYear'].replace(np.NaN,0)
<ipython-input-30-dc3bccf2ab70>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titles['startYear']=titles['startYear'].astype('int64')


In [35]:
titles['startYear']=titles['startYear'].replace(0,np.NaN)

<ipython-input-35-0b7bf8a7141f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titles['startYear']=titles['startYear'].replace(0,np.NaN)


In [45]:
a = pd.merge(movies, titles, left_on='Name', right_on='primaryTitle', how='left')

In [46]:
a

,Movie_ID,Year,Name,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,1,2003,Dinosaur Planet,tvMiniSeries,Dinosaur Planet,Dinosaur Planet,0,2003.0,NaN,"Animation,Documentary,Family"
1,2,2004,Isle of Man TT 2004 Review,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1997,Character,movie,Character,Karakter,0,1997.0,122.0,"Crime,Drama,Mystery"
3,3,1997,Character,tvEpisode,Character,Character,0,2019.0,30.0,Documentary
4,3,1997,Character,short,Character,Character,0,2020.0,17.0,Short
...,...,...,...,...,...,...,...,...,...,...
186360,17770,2003,Alien Hunter,short,Alien Hunter,Alien Hunter,0,2023.0,8.0,"Comedy,Short"
186361,17770,2003,Alien Hunter,short,Alien Hunter,Alien Hunter,0,1984.0,11.0,"Action,Sci-Fi,Short"
186362,17770,2003,Alien Hunter,short,Alien Hunter,Alien Hunter,0,2016.0,NaN,"Drama,Sci-Fi,Short"
186363,17770,2003,Alien Hunter,short,Alien Hunter,Alien Hunter,0,2015.0,15.0,"Sci-Fi,Short"


In [50]:
a.head(40)

,Movie_ID,Year,Name,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,1,2003,Dinosaur Planet,tvMiniSeries,Dinosaur Planet,Dinosaur Planet,0,2003.0,NaN,"Animation,Documentary,Family"
1,2,2004,Isle of Man TT 2004 Review,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1997,Character,movie,Character,Karakter,0,1997.0,122.0,"Crime,Drama,Mystery"
3,3,1997,Character,tvEpisode,Character,Character,0,2019.0,30.0,Documentary
4,3,1997,Character,short,Character,Character,0,2020.0,17.0,Short
5,3,1997,Character,tvEpisode,Character,Character,0,2020.0,NaN,"Action,Reality-TV"
6,3,1997,Character,movie,Character,Kyarakutâ,0,2021.0,125.0,"Crime,Drama,Mystery"
7,3,1997,Character,tvEpisode,Character,Character,0,2014.0,NaN,"Comedy,Documentary,Family"
8,3,1997,Character,short,Character,Character,0,2021.0,28.0,"Sci-Fi,Short"
9,3,1997,Character,short,Character,Character,0,2021.0,39.0,"Documentary,Short"


In [51]:
a = a.drop_duplicates(subset=['Movie_ID'])

In [53]:
a=a.drop(columns=['startYear'])

In [55]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17770 entries, 0 to 186354
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Movie_ID        17770 non-null  int64  
 1   Year            17770 non-null  int64  
 2   Name            17770 non-null  object 
 3   titleType       12099 non-null  object 
 4   primaryTitle    12099 non-null  object 
 5   originalTitle   12099 non-null  object 
 6   isAdult         12099 non-null  object 
 7   runtimeMinutes  11299 non-null  float64
 8   genres          11973 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 1.4+ MB


In [56]:
a = a.dropna()

In [58]:
a.to_csv('/content/drive/MyDrive/CMPE256/256_project/prep.csv',index=False)